In [1]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.4/362.4 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.6 MB/s eta 0:00:00


In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
!setsid nohup ollama serve > run_serve.txt 2>&1 &

In [4]:
!ollama list

NAME	ID	SIZE	MODIFIED 


In [5]:
!setsid ollama run gemma:2b > run_gemma_2b.txt 2>&1 &

## LLM + Prompt

In [7]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.6 MB/s eta 0:00:00


In [8]:
from langchain_community.llms import Ollama

llm = Ollama(model="gemma:2b")

llm.invoke("how can langsmith help with testing?")

'**Langsmith can help with testing in several ways:**\n\n**1. Identifying and tracking test cases:**\n\n* Langsmith can store and manage vast amounts of test cases, including manual and automated tests.\n* It can automatically identify and prioritize test cases based on factors like complexity, dependencies, and risk score.\n* This helps in focusing testing efforts on the most important and impactful areas.\n\n**2. Generating test data:**\n\n* Langsmith can generate synthetic data that mimics real-world scenarios, reducing manual effort and accelerating test execution.\n* It can also generate test cases based on existing data, facilitating the creation of comprehensive coverage for various scenarios.\n\n**3. Analyzing test results and identifying trends:**\n\n* Langsmith provides detailed reports on test execution, highlighting areas of success and failure.\n* These reports help identify potential issues, improve test quality, and guide further testing efforts.\n\n**4. Facilitating com

In [9]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

chain = prompt | llm

chain.invoke({"input": "how can langsmith help with testing?"})

'**Langsmith can help with testing in several ways:**\n\n* **Automated testing:** Langsmith can automatically generate test cases based on your documentation, reducing the manual effort required for writing and maintaining test cases.\n* **Test case documentation:** Langsmith can automatically generate comprehensive documentation for your test cases, making them easy to understand and maintain.\n* **Defect and issue identification:** Langsmith can analyze your documentation and identify potential defects and issues, helping you to prioritize and fix them more effectively.\n* **Test case prioritization:** Langsmith can prioritize your test cases based on their impact and dependencies, allowing you to focus on the most important tests first.\n* **Collaboration:** Langsmith can be easily shared and used by multiple team members, including developers, testers, and product owners, making it easier to collaborate on testing efforts.\n\n**Additionally, Langsmith can:**\n\n* **Generate reports

In [10]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"input": "how can langsmith help with testing?"})

'**Langsmith can help with testing by:**\n\n**1. Identifying potential testing gaps:**\n\n* Langsmith can analyze the documentation to identify areas that are not well tested or that are not covered by existing test cases.\n* By highlighting these gaps, Langsmith can prioritize areas for testing and ensure that all critical functionality is thoroughly tested.\n\n**2. Generating automated test cases:**\n\n* Langsmith can automatically generate test cases based on the documentation.\n* This can save time and effort, allowing testers to start testing more quickly and efficiently.\n\n**3. Identifying potential edge cases and boundary cases:**\n\n* Langsmith can analyze the documentation to identify scenarios that are not covered by existing test cases.\n* These scenarios can be tested using a variety of approaches, such as fuzzing, scenario-based testing, and black-box testing.\n\n**4. Prioritizing testing efforts:**\n\n* Langsmith can rank test cases based on their importance and likeliho

## RAG Retrieval Chain

In [11]:
!pip install beautifulsoup4
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 22.9 MB/s eta 0:00:00


In [12]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()
embeddings = OllamaEmbeddings(model="gemma:2b")
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:
    <context>
    {context}
    </context>
    Question: {input}
    """)

from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)

from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke(
    {"input": "how can langsmith help with testing?"})

print(response["answer"])

Langsmith can help with testing by providing several features and functionalities:

* **Tracing:** Langsmith provides a comprehensive view of all traces from a single conversation, making it easy to track the performance of and annotate your application across multiple turns.
* **Feedback collection:** Langsmith allows you to attach feedback scores to logged traces, which can be filtered and used to identify regressions and improve the application.
* **Annotation:** You can send runs to annotation queues, where annotators can closely inspect interesting traces and annotate them with respect to different criteria.
* **Data collection and analysis:** Langsmith enables you to add runs as examples to datasets, expanding your test coverage on real-world scenarios.
* **Performance monitoring:** Langsmith provides monitoring charts that allow you to track key metrics over time, including latency, cost, and feedback scores.
* **Automatics:** Langsmith offers automation features that allow you 

In [13]:
# small document for comparison
# llm could not answer the question based on the small document

from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)

from langchain_core.documents import Document
document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'The context does not specify how langsmith can help with testing, so I cannot answer this question from the provided context.'

## Conversation Retrieval Chain

In [14]:
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])

from langchain.chains import create_history_aware_retriever

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content="Can LangSmith help test my LLM applications?"),
    AIMessage(content="Yes!")
]

retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.', 'language': 'en'}, page_content='Skip to main contentGo to API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application deve

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])

document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

chat_history = [
    HumanMessage(content="Can LangSmith help test my LLM applications?"),
    AIMessage(content="Yes!")
]

retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

In [15]:
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
])

document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

chat_history = [
    HumanMessage(content="Can LangSmith help test my LLM applications?"),
    AIMessage(content="Yes!")
]

retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.', 'language': 'en'}, page_content='Skip to main contentGo to API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith, which is no longer actively

In [18]:
answer = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})['answer']

print(answer)

Yes, LangSmith can help test your LLM applications. The user can create datasets, run custom evaluations, and filter traces by feedback score to identify regressions and areas for improvement. This allows the user to iteratively refine their application and ensure its quality.
